# Silver Layer NoteBook

##### Mount Datalake 

In [0]:
# configs = {"fs.azure.account.auth.type": "OAuth",
#           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#           "fs.azure.account.oauth2.client.id": "00689b21-a3d9-433e-9cf0-7b372b562098",
#           "fs.azure.account.oauth2.client.secret": "<secret-key>",
#           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/06865a3a-3892-4636-8359-dd415777f2c4/oauth2/token"}


# dbutils.fs.mount(
#   source = "abfss://bronze@mylambdadatalake.dfs.core.windows.net/",
#   mount_point = "/mnt/bronze",
#   extra_configs = configs)


# dbutils.fs.mount(
#   source = "abfss://silver@mylambdadatalake.dfs.core.windows.net/",
#   mount_point = "/mnt/silver",
#   extra_configs = configs)

In [0]:
# Display the mount endpoints
# mounts = dbutils.fs.mounts()
# display(mounts)


### Loding Data Sets

### 1) Claims Dataset

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("Facility Name", StringType(), True),
    StructField("Facility ID", IntegerType(), True),
    StructField("State", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("Claim Type", StringType(), True),
    StructField("Avg Spndg Per EP Hospital", DoubleType(), True),
    StructField("Avg Spndg Per EP State", DoubleType(), True),
    StructField("Avg Spndg Per EP National", DoubleType(), True),
    StructField("Percent of Spndg Hospital", StringType(), True),
    StructField("Percent of Spndg State", StringType(), True),
    StructField("Percent of Spndg National", StringType(), True),
    StructField("Start Date", StringType(), True),
    StructField("End Date", StringType(), True)
])

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
Claims_df = spark.read.format("csv").option("header", "true").schema(schema).load("/mnt/bronze/Claims")

# remove duplicates in Drugs_df
Claims_df = Claims_df.dropDuplicates()

# display(Claims_df.limit(10))

Facility Name,Facility ID,State,Period,Claim Type,Avg Spndg Per EP Hospital,Avg Spndg Per EP State,Avg Spndg Per EP National,Percent of Spndg Hospital,Percent of Spndg State,Percent of Spndg National,Start Date,End Date
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Carrier,779.0,699.0,751.0,2.73%,2.73%,2.82%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Hospice,1.0,2.0,1.0,0.00%,0.01%,0.00%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Inpatient,37.0,39.0,45.0,0.13%,0.15%,0.17%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Durable Medical Equipment,12.0,15.0,13.0,0.04%,0.06%,0.05%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Skilled Nursing Facility,15.0,11.0,16.0,0.05%,0.04%,0.06%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Outpatient,148.0,126.0,190.0,0.52%,0.49%,0.71%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,During Index Hospital Admission,Home Health Agency,0.0,0.0,0.0,0.00%,0.00%,0.00%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,During Index Hospital Admission,Hospice,0.0,0.0,0.0,0.00%,0.00%,0.00%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Home Health Agency,28.0,23.0,19.0,0.10%,0.09%,0.07%,01/01/2023,12/31/2023
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,During Index Hospital Admission,Inpatient,12935.0,11810.0,12235.0,45.30%,46.21%,46.01%,01/01/2023,12/31/2023


In [0]:
# Convert Strat Date and End Date into date type
# Remove "%" and convert percent sign to double type
from pyspark.sql.functions import to_date, regexp_replace, col, round

Claims_df = Claims_df.withColumn("Start Date", to_date(col("Start Date"), "MM/dd/yyyy")) \
                     .withColumn("End Date", to_date(col("End Date"), "MM/dd/yyyy")) \
                     .withColumn(
                         "Percent of Spndg Hospital", 
                         round(regexp_replace(col("Percent of Spndg Hospital"), "%", "").cast("double") / 100, 4)
                     ).withColumn(
                         "Percent of Spndg State", 
                         round(regexp_replace(col("Percent of Spndg State"), "%", "").cast("double") / 100, 4)
                     ).withColumn(
                         "Percent of Spndg National", 
                         round(regexp_replace(col("Percent of Spndg National"), "%", "").cast("double") / 100, 4)
                     )

# display(Claims_df.limit(10))


Facility Name,Facility ID,State,Period,Claim Type,Avg Spndg Per EP Hospital,Avg Spndg Per EP State,Avg Spndg Per EP National,Percent of Spndg Hospital,Percent of Spndg State,Percent of Spndg National,Start Date,End Date
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Carrier,779.0,699.0,751.0,0.0273,0.0273,0.0282,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Hospice,1.0,2.0,1.0,0.0,1.0E-4,0.0,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Inpatient,37.0,39.0,45.0,0.0013,0.0015,0.0017,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Durable Medical Equipment,12.0,15.0,13.0,4.0E-4,6.0E-4,5.0E-4,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Skilled Nursing Facility,15.0,11.0,16.0,5.0E-4,4.0E-4,6.0E-4,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Outpatient,148.0,126.0,190.0,0.0052,0.0049,0.0071,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,During Index Hospital Admission,Home Health Agency,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,During Index Hospital Admission,Hospice,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,1 to 3 days Prior to Index Hospital Admission,Home Health Agency,28.0,23.0,19.0,0.001,9.0E-4,7.0E-4,2023-01-01,2023-12-31
SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,During Index Hospital Admission,Inpatient,12935.0,11810.0,12235.0,0.453,0.4621,0.4601,2023-01-01,2023-12-31


In [0]:
# Extract onlythe Year of the Start Date and make column as Year
from pyspark.sql.functions import year

Claims_df = Claims_df.withColumn("Year", year(col("Start Date")))

# display(Claims_df.limit(5))

Facility Name,Facility ID,State,Period,Claim Type,Avg Spndg Per EP Hospital,Avg Spndg Per EP State,Avg Spndg Per EP National,Percent of Spndg Hospital,Percent of Spndg State,Percent of Spndg National,Start Date,End Date,Year
JACKSON HOSPITAL & CLINIC INC,10024,AL,1 to 3 days Prior to Index Hospital Admission,Carrier,682.0,699.0,751.0,0.0258,0.0273,0.0282,2023-01-01,2023-12-31,2023
GADSDEN REGIONAL MEDICAL CENTER,10040,AL,1 through 30 days After Discharge from Index Hospital Admission,Inpatient,6147.0,4513.0,4100.0,0.2387,0.1766,0.1542,2023-01-01,2023-12-31,2023
ATHENS LIMESTONE HOSPITAL,10079,AL,1 through 30 days After Discharge from Index Hospital Admission,Inpatient,3315.0,4513.0,4100.0,0.1345,0.1766,0.1542,2023-01-01,2023-12-31,2023
ATHENS LIMESTONE HOSPITAL,10079,AL,1 through 30 days After Discharge from Index Hospital Admission,Skilled Nursing Facility,5987.0,3030.0,3944.0,0.243,0.1186,0.1483,2023-01-01,2023-12-31,2023
NORTHWEST MEDICAL CENTER,10086,AL,During Index Hospital Admission,Skilled Nursing Facility,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-01,2023-12-31,2023


In [0]:
# Create binary features for high spending percentages
Claims_df = Claims_df.withColumn("High Spndg Hospital", (col("Percent of Spndg Hospital") > 0.5).cast("int")) \
                     .withColumn("High Spndg State", (col("Percent of Spndg State") > 0.5).cast("int")) \
                     .withColumn("High Spndg National", (col("Percent of Spndg National") > 0.5).cast("int"))

# display(Claims_df.limit(5))

Facility Name,Facility ID,State,Period,Claim Type,Avg Spndg Per EP Hospital,Avg Spndg Per EP State,Avg Spndg Per EP National,Percent of Spndg Hospital,Percent of Spndg State,Percent of Spndg National,Start Date,End Date,Year,High Spndg Hospital,High Spndg State,High Spndg National
JACKSON HOSPITAL & CLINIC INC,10024,AL,1 to 3 days Prior to Index Hospital Admission,Carrier,682.0,699.0,751.0,0.0258,0.0273,0.0282,2023-01-01,2023-12-31,2023,0,0,0
GADSDEN REGIONAL MEDICAL CENTER,10040,AL,1 through 30 days After Discharge from Index Hospital Admission,Inpatient,6147.0,4513.0,4100.0,0.2387,0.1766,0.1542,2023-01-01,2023-12-31,2023,0,0,0
ATHENS LIMESTONE HOSPITAL,10079,AL,1 through 30 days After Discharge from Index Hospital Admission,Inpatient,3315.0,4513.0,4100.0,0.1345,0.1766,0.1542,2023-01-01,2023-12-31,2023,0,0,0
ATHENS LIMESTONE HOSPITAL,10079,AL,1 through 30 days After Discharge from Index Hospital Admission,Skilled Nursing Facility,5987.0,3030.0,3944.0,0.243,0.1186,0.1483,2023-01-01,2023-12-31,2023,0,0,0
NORTHWEST MEDICAL CENTER,10086,AL,During Index Hospital Admission,Skilled Nursing Facility,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-01,2023-12-31,2023,0,0,0


In [0]:
# from pyspark.sql.functions import col
# filtered_df = Claims_df.filter(col("Percent of Spndg Hospital") > 0.5)
# display(filtered_df)

In [0]:
# unique_states = Claims_df.select("State").distinct()
# display(unique_states)

In [0]:
# Claims_df.count()

64658

In [0]:
# from pyspark.sql.functions import col, sum

# # Check for null values in each column
# null_counts = Claims_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in Claims_df.columns])

# display(null_counts)

Facility Name,Facility ID,State,Period,Claim Type,Avg Spndg Per EP Hospital,Avg Spndg Per EP State,Avg Spndg Per EP National,Percent of Spndg Hospital,Percent of Spndg State,Percent of Spndg National,Start Date,End Date,Year,High Spndg Hospital,High Spndg State,High Spndg National
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2) Drugs Dataset

In [0]:
drugs_schema = StructType([
    StructField("Brnd_Name", StringType(), True),
    StructField("Gnrc_Name", StringType(), True),
    StructField("Tot_Mftr", IntegerType(), True),
    StructField("Mftr_Name", StringType(), True),
    StructField("Tot_Spndng_2018", DoubleType(), True),
    StructField("Tot_Dsg_Unts_2018", DoubleType(), True),
    StructField("Tot_Clms_2018", DoubleType(), True),
    StructField("Avg_Spnd_Per_Dsg_Unt_Wghtd_2018", DoubleType(), True),
    StructField("Avg_Spnd_Per_Clm_2018", DoubleType(), True),
    StructField("Outlier_Flag_2018", StringType(), True),
    StructField("Tot_Spndng_2019", DoubleType(), True),
    StructField("Tot_Dsg_Unts_2019", DoubleType(), True),
    StructField("Tot_Clms_2019", DoubleType(), True),
    StructField("Avg_Spnd_Per_Dsg_Unt_Wghtd_2019", DoubleType(), True),
    StructField("Avg_Spnd_Per_Clm_2019", DoubleType(), True),
    StructField("Outlier_Flag_2019", StringType(), True),
    StructField("Tot_Spndng_2020", DoubleType(), True),
    StructField("Tot_Dsg_Unts_2020", DoubleType(), True),
    StructField("Tot_Clms_2020", DoubleType(), True),
    StructField("Avg_Spnd_Per_Dsg_Unt_Wghtd_2020", DoubleType(), True),
    StructField("Avg_Spnd_Per_Clm_2020", DoubleType(), True),
    StructField("Outlier_Flag_2020", StringType(), True),
    StructField("Tot_Spndng_2021", DoubleType(), True),
    StructField("Tot_Dsg_Unts_2021", DoubleType(), True),
    StructField("Tot_Clms_2021", DoubleType(), True),
    StructField("Avg_Spnd_Per_Dsg_Unt_Wghtd_2021", DoubleType(), True),
    StructField("Avg_Spnd_Per_Clm_2021", DoubleType(), True),
    StructField("Outlier_Flag_2021", StringType(), True),
    StructField("Tot_Spndng_2022", DoubleType(), True),
    StructField("Tot_Dsg_Unts_2022", DoubleType(), True),
    StructField("Tot_Clms_2022", DoubleType(), True),
    StructField("Avg_Spnd_Per_Dsg_Unt_Wghtd_2022", DoubleType(), True),
    StructField("Avg_Spnd_Per_Clm_2022", DoubleType(), True),
    StructField("Outlier_Flag_2022", StringType(), True),
    StructField("Chg_Avg_Spnd_Per_Dsg_Unt_21_22", DoubleType(), True),
    StructField("CAGR_Avg_Spnd_Per_Dsg_Unt_18_22", DoubleType(), True)
])

In [0]:
Drugs_df = spark.read.format("csv").option("header", "true").schema(drugs_schema).load("/mnt/bronze/Drugs")

# remove duplicates in Drugs_df
Drugs_df = Drugs_df.dropDuplicates()

# display(Drugs_df.limit(10))

Rename the columns

In [0]:
column_rename_map_Drugs = {
    "Brnd_Name": "Brand_Name",
    "Gnrc_Name": "Generic_Name",
    "Tot_Mftr": "Number_of_Manufacturers",
    "Mftr_Name": "Manufacturer",
    "Tot_Spndng_2018": "2018_Total_Spending",
    "Tot_Dsg_Unts_2018": "2018_Total_Dosage_Units",
    "Tot_Clms_2018": "2018_Total_Claims",
    "Avg_Spnd_Per_Dsg_Unt_Wghtd_2018": "2018_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "Avg_Spnd_Per_Clm_2018": "2018_Average_Total_Spending_Per_Claim",
    "Outlier_Flag_2018": "2018_Outlier_Flag",
    "Tot_Spndng_2019": "2019_Total_Spending",
    "Tot_Dsg_Unts_2019": "2019_Total_Dosage_Units",
    "Tot_Clms_2019": "2019_Total_Claims",
    "Avg_Spnd_Per_Dsg_Unt_Wghtd_2019": "2019_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "Avg_Spnd_Per_Clm_2019": "2019_Average_Total_Spending_Per_Claim",
    "Outlier_Flag_2019": "2019_Outlier_Flag",
    "Tot_Spndng_2020": "2020_Total_Spending",
    "Tot_Dsg_Unts_2020": "2020_Total_Dosage_Units",
    "Tot_Clms_2020": "2020_Total_Claims",
    "Avg_Spnd_Per_Dsg_Unt_Wghtd_2020": "2020_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "Avg_Spnd_Per_Clm_2020": "2020_Average_Total_Spending_Per_Claim",
    "Outlier_Flag_2020": "2020_Outlier_Flag",
    "Tot_Spndng_2021": "2021_Total_Spending",
    "Tot_Dsg_Unts_2021": "2021_Total_Dosage_Units",
    "Tot_Clms_2021": "2021_Total_Claims",
    "Avg_Spnd_Per_Dsg_Unt_Wghtd_2021": "2021_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "Avg_Spnd_Per_Clm_2021": "2021_Average_Total_Spending_Per_Claim",
    "Outlier_Flag_2021": "2021_Outlier_Flag",
    "Tot_Spndng_2022": "2022_Total_Spending",
    "Tot_Dsg_Unts_2022": "2022_Total_Dosage_Units",
    "Tot_Clms_2022": "2022_Total_Claims",
    "Avg_Spnd_Per_Dsg_Unt_Wghtd_2022": "2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "Avg_Spnd_Per_Clm_2022": "2022_Average_Total_Spending_Per_Claim",
    "Outlier_Flag_2022": "2022_Outlier_Flag",
    "Chg_Avg_Spnd_Per_Dsg_Unt_21_22": "Change_in_Average_Total_Spending_Per_Dosage_Unit_(2021-2022)",
    "CAGR_Avg_Spnd_Per_Dsg_Unt_18_22": "Annual_Growth_Rate_in_Average_Total_Spending_Per_Dosage_Unit_(2018-2022)"
}

# Rename the columns using the mapping
for old_col, new_col in column_rename_map_Drugs.items():
    if old_col in Drugs_df.columns:
        Drugs_df = Drugs_df.withColumnRenamed(old_col, new_col)

# display(Drugs_df.limit(10))


Logic for the "Change_in_Average_Total_Spending_Per_Dosage_Unit_(2021-2022)" Column

In [0]:
# percent_change = (current - previous) / previous
#percent_change = (2022 - 2021) / 2021



In [0]:
# Replace null values with 0 below mentioned columns
from pyspark.sql.functions import coalesce, lit

columns_to_replace = [
    "2018_Total_Spending",
    "2019_Total_Spending",
    "2020_Total_Spending",
    "2021_Total_Spending",
    "2022_Total_Spending",
    "2019_Total_Dosage_Units",
    "2019_Total_Claims",
    "2019_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "2019_Average_Total_Spending_Per_Claim",
    "2019_Outlier_Flag",
    "2020_Total_Dosage_Units",
    "2020_Total_Claims",
    "2020_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "2020_Average_Total_Spending_Per_Claim",
    "2020_Outlier_Flag",
    "2021_Total_Dosage_Units",
    "2021_Total_Claims",
    "2021_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "2021_Average_Total_Spending_Per_Claim",
    "2021_Outlier_Flag",
    "2022_Total_Dosage_Units",
    "2022_Total_Claims",
    "2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "2022_Average_Total_Spending_Per_Claim",
    "2022_Outlier_Flag",
    "2018_Total_Dosage_Units",
    "2018_Total_Claims",
    "2018_Average_Total_Spending_Per_Dosage_Unit_(Weighted)",
    "2018_Average_Total_Spending_Per_Claim",
    "2018_Outlier_Flag",
    "Change_in_Average_Total_Spending_Per_Dosage_Unit_(2021-2022)",
    "Annual_Growth_Rate_in_Average_Total_Spending_Per_Dosage_Unit_(2018-2022)"
]

for col_name in columns_to_replace:
    Drugs_df = Drugs_df.withColumn(col_name, coalesce(col(col_name), lit(0)))

# display(Drugs_df.limit(10))

In [0]:
from pyspark.sql import functions as F
# Calculte the Compound Annual Growth Rate (CAGR) formular (2018-2022)
Drugs_df = Drugs_df.withColumn(
    "Annual_Growth_Rate_in_Average_Total_Spending_Per_Dosage_Unit_(2018-2022)",
    F.when(
        (F.col("2018_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0) & (F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0),
        (F.pow((F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") / F.col("2018_Average_Total_Spending_Per_Dosage_Unit_(Weighted)")), 1/4) - 1)
    ).otherwise(
        F.when(
            (F.col("2019_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0) & (F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0),
            (F.pow((F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") / F.col("2019_Average_Total_Spending_Per_Dosage_Unit_(Weighted)")), 1/3) - 1)
        ).otherwise(
            F.when(
                (F.col("2020_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0) & (F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0),
                (F.pow((F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") / F.col("2020_Average_Total_Spending_Per_Dosage_Unit_(Weighted)")), 1/2) - 1)
            ).otherwise(
                F.when(
                    (F.col("2021_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0) & (F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") > 0),
                    (F.col("2022_Average_Total_Spending_Per_Dosage_Unit_(Weighted)") / F.col("2021_Average_Total_Spending_Per_Dosage_Unit_(Weighted)")) - 1
                )
            )
        )
    )
)

In [0]:
Drugs_df = Drugs_df.withColumn(
    "Annual_Growth_Rate_in_Average_Total_Spending_Per_Dosage_Unit_(2018-2022)",
    coalesce(F.col("Annual_Growth_Rate_in_Average_Total_Spending_Per_Dosage_Unit_(2018-2022)"), lit(0))
)


In [0]:
# from pyspark.sql.functions import col, sum

# # Check for null values in each column
# null_counts = Drugs_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in Drugs_df.columns])

# display(null_counts)

### 3) Medicare_DME_Devices_Supplies dataset

In [0]:
Medicare_DME_DS_schema = StructType([
    StructField("ï»¿Rfrg_NPI", IntegerType(), False),
    StructField("Rfrg_Prvdr_Last_Name_Org", StringType(), True),
    StructField("Rfrg_Prvdr_First_Name", StringType(), True),
    StructField("Rfrg_Prvdr_MI", StringType(), True),
    StructField("Rfrg_Prvdr_Crdntls", StringType(), True),
    StructField("Rfrg_Prvdr_Ent_Cd", StringType(), True),
    StructField("Rfrg_Prvdr_St1", StringType(), True),
    StructField("Rfrg_Prvdr_St2", StringType(), True),
    StructField("Rfrg_Prvdr_City", StringType(), True),
    StructField("Rfrg_Prvdr_State_Abrvtn", StringType(), True),
    StructField("Rfrg_Prvdr_State_FIPS", StringType(), True),
    StructField("Rfrg_Prvdr_Zip5", StringType(), True),
    StructField("Rfrg_Prvdr_RUCA_Cat", StringType(), True),
    StructField("Rfrg_Prvdr_RUCA", DoubleType(), True),
    StructField("Rfrg_Prvdr_RUCA_Desc", StringType(), True),
    StructField("Rfrg_Prvdr_Cntry", StringType(), True),
    StructField("Rfrg_Prvdr_Spclty_Cd", DoubleType(), True),
    StructField("Rfrg_Prvdr_Spclty_Desc", StringType(), True),
    StructField("Rfrg_Prvdr_Spclty_Srce", StringType(), True),
    StructField("RBCS_Lvl", StringType(), True),
    StructField("RBCS_Id", StringType(), True),
    StructField("RBCS_Desc", StringType(), True),
    StructField("HCPCS_CD", StringType(), True),
    StructField("HCPCS_Desc", StringType(), True),
    StructField("Suplr_Rentl_Ind", StringType(), True),
    StructField("Tot_Suplrs", DoubleType(), True),
    StructField("Tot_Suplr_Benes", DoubleType(), True),
    StructField("Tot_Suplr_Clms", DoubleType(), True),
    StructField("Tot_Suplr_Srvcs", DoubleType(), True),
    StructField("Avg_Suplr_Sbmtd_Chrg", StringType(), True),
    StructField("Avg_Suplr_Mdcr_Alowd_Amt", StringType(), True),
    StructField("Avg_Suplr_Mdcr_Pymt_Amt", StringType(), True),
    StructField("Avg_Suplr_Mdcr_Stdzd_Amt", StringType(), True)
])



In [0]:
Medicare_DME_DS_df = spark.read.format("csv").option("header", "true").schema(Medicare_DME_DS_schema).load("/mnt/bronze/Medicare_DME_Devices_Supplies/")

# remove duplicates in Medicare_DME_DS_df
Medicare_DME_DS_df = Medicare_DME_DS_df.dropDuplicates()

# display(Medicare_DME_DS_df.limit(5))

In [0]:
# # Calculate no of column in Medicare_DME_DS_df 
# num_columns = len(Medicare_DME_DS_df.columns)
# num_columns

Rename Columns

In [0]:
column_rename_map_Medicare_DME_DS = {
    "ï»¿Rfrg_NPI": "Referring_NPI",
    "Rfrg_Prvdr_Last_Name_Org": "Referring_Provider_Last_Name_/_Organization_Name",
    "Rfrg_Prvdr_First_Name": "Referring_Provider_First_Name",
    "Rfrg_Prvdr_MI": "Referring_Provider_Middle_Initial",
    "Rfrg_Prvdr_Crdntls": "Referring_Credentials",
    "Rfrg_Prvdr_Ent_Cd": "Referring_Entity_Code",
    "Rfrg_Prvdr_St1": "Referring_Provider_Street_1",
    "Rfrg_Prvdr_St2": "Referring_Provider_Street_2",
    "Rfrg_Prvdr_City": "Referring_Provider_City",
    "Rfrg_Prvdr_State_Abrvtn": "Referring_Provider_State",
    "Rfrg_Prvdr_State_FIPS": "Referring_Provider_State_FIPS_Code",
    "Rfrg_Prvdr_Zip5": "Referring_Provider_ZIP",
    "Rfrg_Prvdr_RUCA_CAT": "Referring_Provider_RUCA_Category",
    "Rfrg_Prvdr_RUCA": "Referring_Provider_RUCA",
    "Rfrg_Prvdr_RUCA_Desc": "Referring_Provider_RUCA_Description",
    "Rfrg_Prvdr_Cntry": "Referring_Provider_Country",
    "Rfrg_Prvdr_Spclty_Cd": "Referring_Provider_Specialty_Code",
    "Rfrg_Prvdr_Spclty_Desc": "Referring_Provider_Specialty_Description",
    "Rfrg_Prvdr_Spclty_Srce": "Referring_Provider_Specialty_Source",
    "RBCS_Lvl": "RBCS_Level",
    "RBCS_Id": "RBCS_ID",
    "RBCS_Desc": "RBCS_Description",
    "HCPCS_CD": "HCPCS_Code",
    "HCPCS_Desc": "HCPCS_Description",
    "Suplr_Rentl_Ind": "Supplier_Rental_Indicator",
    "Tot_Suplrs": "Number_of_Suppliers",
    "Tot_Suplr_Benes": "Number_of_Supplier_Beneficiaries",
    "Tot_Suplr_Clms": "Number_of_Supplier_Claims",
    "Tot_Suplr_Srvcs": "Number_of_Supplier_Services",
    "Avg_Suplr_Mdcr_Alowd_Amt": "Average_Supplier_Medicare_Allowed_Amount",
    "Avg_Suplr_Mdcr_Pymt_Amt": "Average_Supplier_Medicare_Payment_Amount",
    "Avg_Suplr_Mdcr_Stdzd_Amt": "Average_Supplier_Medicare_Standard_Payment_Amount",
    "Avg_Suplr_Sbmtd_Chrg": "Average_Supplier_Submitted_Charges"
}

# Rename the columns using the mapping
for old_col, new_col in column_rename_map_Medicare_DME_DS.items():
    if old_col in Medicare_DME_DS_df.columns:
        Medicare_DME_DS_df = Medicare_DME_DS_df.withColumnRenamed(old_col, new_col)

In [0]:
# Medicare_DME_DS_df.count()

In [0]:
# Remove dollar sign and comma from the below mentioned columns and convert into double type
from pyspark.sql.functions import regexp_replace, col

columns_to_clean = [
    "Average_Supplier_Medicare_Allowed_Amount",
    "Average_Supplier_Medicare_Payment_Amount",
    "Average_Supplier_Medicare_Standard_Payment_Amount",
    "Average_Supplier_Submitted_Charges"
]

for column in columns_to_clean:
    Medicare_DME_DS_df = Medicare_DME_DS_df.withColumn(column, regexp_replace(col(column), "[\\$,]", "").cast("double"))


In [0]:
# in Medicare_DME_DS_df below columns contain null vlues replace with 0

from pyspark.sql.functions import coalesce, lit

columns_to_replace_nulls = [
    "Number_of_Supplier_Beneficiaries",
    "Number_of_Suppliers",
    "Number_of_Supplier_Claims",
    "Number_of_Supplier_Services",
    "Average_Supplier_Medicare_Allowed_Amount",
    "Average_Supplier_Medicare_Payment_Amount",
    "Average_Supplier_Medicare_Standard_Payment_Amount"
]

for column in columns_to_replace_nulls: 
    Medicare_DME_DS_df = Medicare_DME_DS_df.withColumn(column, coalesce(col(column), lit(0)))

# display(Medicare_DME_DS_df.limit(5))

In [0]:
# Check whether column contain non numerical values

# from pyspark.sql.functions import col

# # This regex matches valid numbers including decimals (e.g., 123, 12.34)
# valid_number_pattern = r'^\d+(\.\d+)?$'

# # Filter rows that do NOT match the valid numeric format
# non_numeric_zip_df = Medicare_DME_DS_df.filter(~col("Referring_Provider_Specialty_Code").rlike(valid_number_pattern))

# # Display rows that are NOT valid numbers
# display(non_numeric_zip_df)


In [0]:
# from pyspark.sql.functions import col, sum

# # Calculate the null count for each column
# null_counts = Medicare_DME_DS_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in Medicare_DME_DS_df.columns])

# # Display the null counts
# display(null_counts)

In [0]:
# # Filter rows where Referring_Provider_RUCA column contains null values
# null_referring_provider_ruca_df = Medicare_DME_DS_df.filter(col("Number_of_Suppliers").isNull())

# # Display the filtered rows
# display(null_referring_provider_ruca_df)

## Load the transform data into silver container

In [0]:
# Load Claims_df into silver container 

Claims_df.write.format("parquet")\
.mode("overwrite")\
.option("path","/mnt/silver/Claims")\
.save()

In [0]:
# Load Drugs_df into silver container 

Drugs_df.write.format("parquet")\
.mode("overwrite")\
.option("path","/mnt/silver/Drugs")\
.save()

In [0]:
# Load Medicare_DME_DS_df into silver container 

Medicare_DME_DS_df.write.format("parquet")\
.mode("overwrite")\
.option("path","/mnt/silver/Medicare_DME_DS")\
.save()